In [1]:
import os
# DATA_PATH = "../data/" 
DATA_PATH = "/kaggle/input/" 
SAVE_PATH = "/kaggle/working/"

In [2]:
DATASETS_PATH = os.path.join(DATA_PATH, "nlp-ua-locations-extractions")

In [3]:
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Data

### Load and sample data

In [9]:
uk_geo_dataset = pd.read_csv(os.path.join(DATASETS_PATH, 'uk_geo_dataset.csv'), converters={"loc_markers": eval})

In [10]:
_, uk_geo_dataset_light = train_test_split(uk_geo_dataset, stratify=uk_geo_dataset["is_valid"], test_size=0.2, random_state=42)
val_ratio = (uk_geo_dataset_light['is_valid'] == 1).sum() / uk_geo_dataset_light.shape[0]

In [11]:
uk_geo_dataset_light.sample(5)

,text,loc_markers,org_markers,per_markers,is_valid
541203,"Всесвітній банк заявив, що комахи можуть обійт...","[(50, 64), (67, 72)]",[],[],0
1007506,"Упевнений, що Філ Фоден (хавбек Ман Сіті - при...","[(36, 40)]","[(32, 35)]","[(14, 23), (106, 113)]",1
890123,Міністр доріг і міського розвитку Ірану прибув...,"[(50, 55)]",[],"[(34, 39)]",0
200861,Але для нас це важливий сигнал готовності ЄС д...,"[(70, 75)]","[(42, 44)]",[],0
394969,Київська міська державна адміністрація має нам...,"[(98, 117)]",[],[],0


In [12]:
uk_geo_dataset_light[['text', 'loc_markers', 'is_valid']].to_csv(os.path.join(SAVE_PATH, "uk_geo_dataset_md.csv"), index=None)

In [13]:
df_locations = pd.read_csv(os.path.join(SAVE_PATH, "uk_geo_dataset_md.csv"), converters={'loc_markers': eval})

### Preprocess

In [14]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

In [15]:
import spacy

from spacy.training.iob_utils import biluo_to_iob, doc_to_biluo_tags
from tqdm.autonotebook import tqdm
tqdm.pandas()

df_locations.loc_markers = df_locations.loc_markers.apply(lambda x: [[y[0], y[1], 'LOC']  for y in x])

nlp = spacy.blank("xx")

def convert_to_conll(row):
    data = {
        "text": row['text'],
        "label": row['loc_markers']
    }
    doc = nlp(data["text"])
    ents = []
    for start, end, label in data["label"]:
        span = doc.char_span(start, end, label=label, alignment_mode='contract')
        if span is None:
            span_extended = doc.char_span(start, end, label=label, alignment_mode='expand')
            print(f"Skipping span {span_extended} expanded from {doc.text[start:end]}")
        else:
            ents.append(span)
    doc.ents = ents
    return {
        'tokens': list([t.text for t in doc]),
        'labels': list(biluo_to_iob(doc_to_biluo_tags(doc)))
    }

df_locations['conll'] = df_locations.progress_apply(convert_to_conll, axis=1)

  0%|          | 0/202000 [00:00<?, ?it/s]

In [16]:
label2id = {'O': 0, 'B-LOC': 1, 'I-LOC': 2}

df_locations['tokens'] = df_locations.conll.str['tokens']
df_locations['ner_tags'] = df_locations.conll.str['labels'].apply(lambda x: [label2id[t] for t in x])

df_train = df_locations[df_locations.is_valid == 0]
df_valid = df_locations[df_locations.is_valid == 1]

In [17]:
df_train[['tokens', 'ner_tags']].to_json(
    os.path.join(SAVE_PATH, 'train_uk_light.json'), orient='records', lines=True)
df_valid[['tokens', 'ner_tags']].to_json(
    os.path.join(SAVE_PATH, 'valid_uk_light.json'), orient='records', lines=True)

In [18]:
import datasets

In [19]:
from datasets import load_dataset

raw_datasets = load_dataset(
    "json",
    data_files={
        'train': os.path.join(SAVE_PATH, 'train_uk_light.json'),
        'val': os.path.join(SAVE_PATH, 'valid_uk_light.json'),
    },
)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-3b3b5c9cf2f3f467/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

label2id = {'O': 0, 'B-LOC': 1, 'I-LOC': 2}
id2label = {v: k for k, v in label2id.items()}

model = AutoModelForTokenClassification.from_pretrained(
    'youscan/ukr-roberta-base',
#     'bert-base-multilingual-cased',
    id2label=id2label,
    label2id=label2id,
)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at youscan/ukr-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
# tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenizer = AutoTokenizer.from_pretrained('youscan/ukr-roberta-base', add_prefix_space=True)

In [22]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id is None:
            new_labels.append(-100)
        elif word_id != current_word:
            # Start of a new word!
            current_word = word_id
            new_labels.append(labels[word_id])
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [23]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

  0%|          | 0/200 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [24]:
from transformers import TrainingArguments

args = TrainingArguments(
    "roberta-ua-loc-ner",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3
)

In [25]:
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW

optimizer = AdamW([
    {'params': list(model.roberta.parameters()), 'lr': 1e-5},
    {'params': list(model.classifier.parameters()), 'lr': 1e-3}
])

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0.1*3*(tokenized_datasets['train'].num_rows/16),
    num_training_steps=3*(tokenized_datasets['train'].num_rows/16)
)

In [26]:
from typing import List

def metrics(y_true: List[List[str]], y_pred: List[List[str]]):
    assert len(y_true) == len(y_pred)
    tp, fp, fn = 0.0, 0.0, 0.0

    for y_true_sample, y_pred_sample in zip(y_true, y_pred):
        tp += len(set(y_true_sample) & set(y_pred_sample))
        fp += len(set(y_pred_sample) - set(y_true_sample))
        fn += len(set(y_true_sample) - set(y_pred_sample))
    
    precision = tp / (tp + fp) if tp + fp != 0 else 0.0 if tp + fn != 0.0 else 1.0
    recall = tp / (tp + fn) if tp + fn != 0 else 1.0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall != 0 else 0.0

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [27]:
# import evaluate
import numpy as np

# metric = evaluate.load("seqeval")

label_names = list(label2id.keys())

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    return metrics(y_true=true_predictions, y_pred=true_labels)

    
#     all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
#     return {
#         "precision": all_metrics["overall_precision"],
#         "recall": all_metrics["overall_recall"],
#         "f1": all_metrics["overall_f1"],
#         "accuracy": all_metrics["overall_accuracy"],
#     }

In [28]:
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("wandb_api")
    wandb.login(key=api_key)
    anony = None
except:
    anony = "must"
    print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [29]:
from transformers import Trainer

train_sample = [tokenized_datasets['train'][i] for i in range(10)]
val_sample = [tokenized_datasets['val'][i] for i in range(10)]

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
#     train_dataset=train_sample,
    eval_dataset=tokenized_datasets["val"],
#     eval_dataset=val_sample,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    optimizers=(optimizer, scheduler)
)

In [30]:
trainer.train()
wandb.finish()

wandb: Currently logged in as: yevhenii-azarov. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.9
wandb: Run data is saved locally in /kaggle/working/wandb/run-20231026_215920-xgnno3mv
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run driven-monkey-5
wandb: ⭐️ View project at https://wandb.ai/yevhenii-azarov/huggingface
wandb: 🚀 View run at https://wandb.ai/yevhenii-azarov/huggingface/runs/xgnno3mv
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.014500,0.020376,0.984079,0.984079,0.984079
2,0.010700,0.018212,0.986353,0.985233,0.985793
3,0.005900,0.021576,0.985216,0.985590,0.985403


wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:                        eval/f1 ▁█▆
wandb:                      eval/loss ▆▁█
wandb:                 eval/precision ▁█▅
wandb:                    eval/recall ▁▆█
wandb:                   eval/runtime █▁▁
wandb:        eval/samples_per_second ▁██
wandb:          eval/steps_per_second ▁██
wandb:                    train/epoch ▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
wandb:              train/global_step ▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
wandb:            train/learning_rate ▂▃▅▇███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:                     train/loss █▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:               train/total_flos ▁
wandb:               train/train_loss ▁
wandb:            train/train_runtime ▁
wandb: train/train_samples_per_second ▁
wandb:   train/train_steps_per_second ▁
wandb: 
wandb: Run summary:
wandb:                        eval/f1 0.9854
wandb:                      eval/l